In [1]:
from AOC_utils import get_day
import numpy as np

day = 24
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Day 24 input already downloaded


['380596900441035, 475034410013298, 238677466991589 @ -141, -244, 154',
 '233796913851006, 262774170759556, 265925724673108 @ 54, 10, 23',
 '276006064958748, 296055609314709, 391999646036593 @ 14, 21, 24',
 '293483465377495, 295344217447825, 424521127946914 @ -5, 21, -13',
 '293267724517878, 460174630326339, 283123920222676 @ -56, -349, 17',
 '109629738653787, 228051835888962, 65687603071905 @ 261, 86, 408',
 '439102647267747, 292512150779892, 283500442341031 @ -257, -11, 67']

In [2]:
example = '''
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
'''.split('\n')[1:-1]

In [3]:
len(input_data)

300

In [34]:
# import shapely

In [4]:
import plotly
import plotly.express as px

In [11]:
def parse_data(data):

    hailstone_positions = [[int(x) for x in line.split('@')[0].replace(' ', '').split(',')] for line in data]
    hailstone_velocities = [[int(x) for x in line.split('@')[1].replace(' ', '').split(',')] for line in data]
    hailstones = np.array([hailstone_positions, hailstone_velocities]).transpose(1, 0, 2).astype(np.float64)

    def get_line_ends(hailstone):
        coord_positions, coord_velocitys = hailstone

        line_ends = []
        position = coord_positions[-1]
        velocity = coord_velocitys[-1]
        # line_end = position + (velocity * t)
        t = hailstones.max()
        return [position + (velocity.astype(np.float64) * t) for position, velocity in zip(coord_positions, coord_velocitys)]
    
    line_ends = [get_line_ends(hailstone) for hailstone in hailstones]

    hailstone_lines = np.array([hailstones[:, 0], line_ends]).transpose(1, 0, 2)
    print(hailstone_lines.shape)
    
    if len(hailstones) < 20:
        fig = px.line_3d()
        for i in range(len(hailstones)):
            fig.add_trace(px.line_3d(x=hailstone_lines[i, :, 0], 
                                    y=hailstone_lines[i, :, 1], 
                                    z=hailstone_lines[i, :, 2]).data[0])
        fig.update_layout(template='plotly_dark')
        fig.show()

    from z3.z3 import Solver, sat, Reals

    x, y, z, vx, vy, vz = Reals('x y z vx vy vz')
    t0, t1, t2, t3 = Reals('t0 t1 t2, t3')
    times = [t0, t1, t2, t3]
    eqs = []
    for i in range(4): # len(hailstones)
        hailstone = hailstones[i]
        eqs.append(x + (vx * times[i]) == hailstone[0][0] + (hailstone[1][0] * times[i]))
        eqs.append(y + (vy * times[i]) == hailstone[0][1] + (hailstone[1][1] * times[i]))
        eqs.append(z + (vz * times[i]) == hailstone[0][2] + (hailstone[1][2] * times[i]))

    s = Solver()
    s.add(eqs)

    if s.check() == sat:
        model = s.model()

        vals = [model[x], model[y], model[z], model[vx], model[vy], model[vz]]
        vals = [val.as_long() for val in vals]
        print(vals)

        total2 = vals[0] + vals[1] + vals[2]
        print("Part 2:", total2)

In [10]:
parse_data(input_data)

(300, 2, 3)
[229429688799267, 217160931330282, 133453231437025, 63, 104, 296]
Part 2: 580043851566574


In [7]:
parse_data(example)

(5, 2, 3)


[24, 13, 10, -3, 1, 2]
Part 2: 47
